<hr>


<span style="font-size:36px"><b>Capacity Planning Design for Mapleleaf</b></span><br><br>
<br>
<span style="font-size:28px"><b>Integer Linear Programming for Optimal Solution</b></span><br>
<br>
<span style="font-size:18px">Jiehui Lin, MSBA</span><br>
<br>
<span style="font-size:18px">[GitHub](https://github.com/JiehuiL906)</span><br>
<br>
<span style="font-size:18px"><b>Hult International Business School</b><span><br>

    
    
<hr>
<hr>


<b> Background </b><br>
Mapleleaf Corporation, a midsized manufacturer and distributor of paper products, anticipated that market demand would outstrip it current capacity 10,000 cartons per day within 5 years (see exhibit 1). It seek to determine whether to open new factory in Guadalajara Mexico to meet the growing products demand. The production cost and daily capacity vary among current four production facilities (see exhibit 2). And the distribution cost from each factory (including proposed Guadalajara plant) to each distribution center displays on exhibit 3, as well as individual distribution center daily demand in both 5-year and 10-year landscape. The Guadalajara plant is projected 4000 cartons daily capacity which takes into account capacity cushion 1000 cartons per day. It takes two years to construct Guadalajara factory and the initial investment is estimated $30 million, straight line depreciation, 10 year useful-life without salvage value, and 300 days per year operation schedule.  

<br>
<b>Methodology and Assumption</b><br>
<ul>
<li>Integer Linear Programming to minimize production and distribution cost totally</li>
<li>the daily fixed cost from initial investment on Guadalajara factory is USD30,000,000 / 10 years / 300 days = USD10,000</li>
<li> Examine scenarios based on both full capacity and 90pct capacity since current four production facilities operate not above 90pct capacity on an ongoing basis</li>
<li> Compare total costs with and without Guadalajara factory</li>
<li> Check optimal production schedule for 10 year demand besides that for 5-years demand
</ul>

<hr>
Case Study Source: O.Hall,Jr. & C. McPeak (2008). Designing Optimal Capacity Planning Strategies. Ivey ID:9B08D033. London, Canada: Ivey Publishing

<img src='exhibit_1.png'>
<img src='exhibit_2.png'>

<img src='exhibit_3.png'>

<span style='font-size:24px'><b>Import Package and Create Lists and Dictionaries for Tables above </b></span>

In [2]:
from pulp import*
import pandas as pd
import numpy as np

# Creates a list of all the factories nodes
Plants = ['1', '2', '3', '4', '5']

# Creates a dictionary for the number of cartons each factory can produce daily
Capacity = {'1': 2500, '2': 1500, '3': 3500, '4': 2500, '5': 4000}

# Creates a dictionary for the production cost of each carton in each factory
Prod_costs = {'1': 14, '2': 19, '3': 13, '4': 17, '5': 10}


# Creates a list of all distribution centers nodes
DC = ['1', '2', '3', '4', '5', '6', '7']

# Creates a dictionary for the number of cartons demanded in each DC in 5-Year

Demand_5 = {'1': 1000,'2': 750,'3': 2500,'4': 1500, '5': 1500, '6': 750, '7': 2000}

# Creates a dictionary for the number of cartons demanded in each DC in 10-Year

Demand_10 = {'1': 1000,'2': 1000,'3': 3000,'4': 2000, '5': 2000, '6': 1000, '7': 3000}

# creates a dictionary for distrubution cost from each factory to each distribution center

distr_costs = {'1': {'1':0.75, '2':2.50, '3':4.50, '4':4.75, '5':1.50, '6':3.00, '7': 5.25},
               '2': {'1':2.50, '2':1.00, '3':2.50, '4':2.75, '5':1.50, '6':2.25, '7': 3.25},
               '3': {'1':4.50, '2':2.50, '3':0.50, '4':2.25, '5':3.75, '6':3.00, '7': 1.75},
               '4': {'1':4.75, '2':2.75, '3':2.25, '4':0.75, '5':2.50, '6':3.50, '7': 3.75},
               '5': {'1':5.25, '2':3.25, '3':1.75, '4':2.50, '5':3.75, '6':3.50, '7': 0.50}}



<span style='font-size: 24px'><b>Scenario 1</b></span><br>
<b>*Run Full Capacity in current facilities <mark>WITHOUT</mark> Guadalajara Factory for the Fifth Year Demand*</b>

In [8]:
# Creates the prob variable to contain the problem data
prob_1 = LpProblem("CP_Problem_1", LpMinimize)
 
# Creates a list of tuples containing all the possible routes from factories to Distribution Centers
Routes_0 = [(p,d) for p in Plants[:-1] for d in DC]

# A dictionary called route_vars_0 is created to contain the referenced variables (the routes)
route_vars_0 = LpVariable.dicts("Route_0",(Plants[:-1],DC),0,None,LpInteger)


# The objective function is added to prob first
prob_1 += lpSum([route_vars_0[p][d]*distr_costs[p][d] for (p,d) in Routes_0]
              + [route_vars_0[p][d]*Prod_costs[p] for(p,d) in Routes_0])


# The capacity maximum constraints are added to prob for each supply node 
for p in Plants[:-1]:
   
    prob_1 += lpSum([route_vars_0[p][d] for d in DC]) <= Capacity[p]



#The demand minimum constraints are added to prob for each demand node 
for d in DC:
    prob_1 += lpSum([route_vars_0[p][d] for p in Plants[:-1]]) == Demand_5[d]



# solve the model
prob_1.solve()
print ('Status:', LpStatus[prob_1.status])


# print objective function value
obj_1 = value(prob_1.objective)
print("The minimum total cost is: ${}".format(round(obj_1,2)))


# Print optimal supply capacity from each factory to distribution center
v_soln_1 = np.array([[round(route_vars_0[p][d].varValue) for d in DC] for p in Plants[:-1]])
v_soln_df_1 = pd.DataFrame(v_soln_1)
v_soln_df_1.columns=['Toronto', 'Kansas City', 'Los Angeles', 'Seattle', 'Chicago', 'Atlanta', 'Guadalajara']
v_soln_df_1.index=['Toronto', 'Kansas City','Los Angeles', 'Seattle']
v_soln_df_1

Status: Optimal
The minimum total cost is: $164562.5


,Toronto,Kansas City,Los Angeles,Seattle,Chicago,Atlanta,Guadalajara
Toronto,1000,0,0,0,1500,0,0
Kansas City,0,750,0,0,0,750,0
Los Angeles,0,0,1500,0,0,0,2000
Seattle,0,0,1000,1500,0,0,0


<span style='font-size:24px'><b>Scenario 2</b></span><br>
<b>*Run 90% Capacity in current facilities <mark>WITHOUT</mark> Guadalajara Factory for the Fifth Year Demand*</b>

In [13]:
# Creates the prob variable to contain the problem data
prob_2 = LpProblem("CP_Problem_2", LpMinimize)
 
# Creates a list of tuples containing all the possible routes from factories to Distribution Centers
Routes_0 = [(p,d) for p in Plants[:-1] for d in DC]

# A dictionary called route_vars_0 is created to contain the referenced variables (the routes)
route_vars_0 = LpVariable.dicts("Route_0",(Plants[:-1],DC),0,None,LpInteger)


# The objective function is added to prob first
prob_2 += lpSum([route_vars_0[p][d]*distr_costs[p][d] for (p,d) in Routes_0]
              + [route_vars_0[p][d]*Prod_costs[p] for(p,d) in Routes_0])


# The capacity maximum constraints are added to prob for each supply node 
for p in Plants[:-1]:
   
    prob_2 += lpSum([route_vars_0[p][d] for d in DC]) <= Capacity[p]*0.9



#The demand minimum constraints are added to prob for each demand node 
for d in DC:
    prob_2 += lpSum([route_vars_0[p][d] for p in Plants[:-1]]) == Demand_5[d]


# solve the model
prob_2.solve()
print ('Status:', LpStatus[prob_2.status])

# print objective function value
obj_2 = value(prob_2.objective)
print("The minimum total cost is: ${}".format(round(obj_2,2)))


# Print optimal supply capacity from each factory to distribution center
v_soln_2 = np.array([[round(route_vars_0[p][d].varValue) for d in DC] for p in Plants[:-1]])
v_soln_df_2 = pd.DataFrame(v_soln_2)
v_soln_df_2.columns=['Toronto', 'Kansas City', 'Los Angeles', 'Seattle', 'Chicago', 'Atlanta', 'Guadalajara']
v_soln_df_2.index=['Toronto', 'Kansas City','Los Angeles', 'Seattle']
v_soln_df_2

Status: Infeasible
The minimum total cost is: $168875.0


,Toronto,Kansas City,Los Angeles,Seattle,Chicago,Atlanta,Guadalajara
Toronto,1000,0,0,0,1250,0,0
Kansas City,0,750,0,0,250,750,600
Los Angeles,0,0,1750,0,0,0,1400
Seattle,0,0,750,1500,0,0,0


Solver returns meaningless infeasible solution with violation of one capacity constraint. Total production from Kansas City is 2350 cartons(750+250+750+600) is much more than capacity 1350 cartons(1500* 90pct), while no more than 90pct capacity should be exceeded. The explanation for the failure to satisfy all the constraints simultaneously is that total supply would largely fall short of the forecasting demand in the fifth year if all current facilities run no more than 90% capacity without any new factory.

<span style='font-size:24px'><b>Scenario 3</b></span><br>
<b>*Run full Capacity in current facilities <mark>WITH</mark> Guadalajara Factory for the Fifth Year Demand*</b>

In [18]:


# Creates the prob variable to contain the problem data
prob_3 = LpProblem("CP_Problem_3", LpMinimize)
 
# Creates a list of tuples containing all the possible routes from factories to Distribution Centers
Routes = [(p,d) for p in Plants for d in DC]

# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route",(Plants,DC),0,None,LpInteger)


# Set up binary variable whether to open factory in Guadalajara
g_vars_3 = LpVariable('gf', cat='Binary' ) 

# The objective function is added to prob first
prob_3 += lpSum([route_vars[p][d]*distr_costs[p][d] for (p,d) in Routes]
              + [route_vars[p][d]*Prod_costs[p] for(p,d) in Routes]
              + [g_vars_3* 10000] ) # fixed cost per day if opening Guadalajara factory


# The capacity maximum constraints are added to prob for each supply node 
for p in Plants[:-1]:
   
    prob_3 += lpSum([route_vars[p][d] for d in DC]) <= Capacity[p]


prob_3 += lpSum([route_vars['5'][d] for d in DC]) <= Capacity['5']*g_vars_3

#The demand minimum constraints are added to prob for each demand node
for d in DC:
    prob_3 += lpSum([route_vars[p][d] for p in Plants]) == Demand_5[d]



# solve the model
prob_3.solve()
print ('Status:', LpStatus[prob_3.status])

# print objective function value
obj_3 = value(prob_3.objective)
print("The minimum total cost is: ${}".format(round(obj_3,2)))

# print binary value whether to open factory in Guadalajara (1 = Open; 0 = Not Open)
print('Whether to open Guadalajara factory:', round(g_vars_3.varValue))

# Print optimal QTY from each factory to distribution center
v_soln_3 = np.array([[round(route_vars[p][d].varValue) for d in DC] for p in Plants])
v_soln_df_3 = pd.DataFrame(v_soln_3)
v_soln_df_3.columns=['Toronto', 'Kansas City', 'Los Angeles', 'Seattle', 'Chicago', 'Atlanta', 'Guadalajara']
v_soln_df_3.index=['Toronto', 'Kansas City','Los Angeles', 'Seattle', 'Guadalajara']
v_soln_df_3



Status: Optimal
The minimum total cost is: $143875.0
Whether to open Guadalajara factory: 1


,Toronto,Kansas City,Los Angeles,Seattle,Chicago,Atlanta,Guadalajara
Toronto,1000,0,0,0,1500,0,0
Kansas City,0,0,0,0,0,0,0
Los Angeles,0,750,2500,0,0,250,0
Seattle,0,0,0,0,0,0,0
Guadalajara,0,0,0,1500,0,500,2000


<span style='font-size:24px'><b>Scenario 4</b></span><br>
<b>*Run 90% Capacity in current facilities <mark>WITH</mark> Guadalajara Factory for the Fifth Year Demand*</b>

In [20]:
# Creates the prob variable to contain the problem data
prob_4 = LpProblem("CP_Problem_4", LpMinimize)
 
# Creates a list of tuples containing all the possible routes from factories to Distribution Centers
Routes = [(p,d) for p in Plants for d in DC]

# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route",(Plants,DC),0,None,LpInteger)


# Set up binary variable whether to open factory in Guadalajara
g_vars_4 = LpVariable('gf', cat='Binary' ) 

# The objective function is added to prob first
prob_4 += lpSum([route_vars[p][d]*distr_costs[p][d] for (p,d) in Routes]
              + [route_vars[p][d]*Prod_costs[p] for(p,d) in Routes]
              + [g_vars_4* 10000] ) # fixed cost per day if opening Guadalajara factory


# The capacity maximum constraints are added to prob for each supply node 
for p in Plants[:-1]:
   
    prob_4 += lpSum([route_vars[p][d] for d in DC]) <= Capacity[p]*0.9


prob_4 += lpSum([route_vars['5'][d] for d in DC]) <= Capacity['5']*g_vars_4

#The demand minimum constraints are added to prob for each demand node
for d in DC:
    prob_4 += lpSum([route_vars[p][d] for p in Plants]) == Demand_5[d]


# solve the model
prob_4.solve()
print ('Status:', LpStatus[prob_4.status])

# print objective function value
obj_4 = value(prob_4.objective)
print("The minimum total cost is: ${}".format(round(obj_4,2)))

# print binary value whether to open factory in Guadalajara (1 = Open; 0 = Not Open)
print('Whether to open Guadalajara factory:', round(g_vars_4.varValue))

# Print optimal QTY from each factory to distribution center
v_soln_4 = np.array([[round(route_vars[p][d].varValue) for d in DC] for p in Plants])
v_soln_df_4 = pd.DataFrame(v_soln_4)
v_soln_df_4.columns=['Toronto', 'Kansas City', 'Los Angeles', 'Seattle', 'Chicago', 'Atlanta', 'Guadalajara']
v_soln_df_4.index=['Toronto', 'Kansas City','Los Angeles', 'Seattle', 'Guadalajara']
v_soln_df_4


Status: Optimal
The minimum total cost is: $145737.5
Whether to open Guadalajara factory: 1


,Toronto,Kansas City,Los Angeles,Seattle,Chicago,Atlanta,Guadalajara
Toronto,1000,0,0,0,1250,0,0
Kansas City,0,0,0,0,0,0,0
Los Angeles,0,650,2500,0,0,0,0
Seattle,0,0,0,600,0,0,0
Guadalajara,0,100,0,900,250,750,2000


<span style='font-size:24px'><b>Scenario 5</b></span><br>
<b>*Run full Capacity in current facilities <mark>WITH</mark> Guadalajara Factory for the <mark>Tenth</mark> Year Demand*</b>

In [22]:
# Creates the prob variable to contain the problem data
prob_5 = LpProblem("CP_Problem_5", LpMinimize)
 
# Creates a list of tuples containing all the possible routes from factories to Distribution Centers
Routes = [(p,d) for p in Plants for d in DC]

# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route",(Plants,DC),0,None,LpInteger)


# Set up binary variable whether to open factory in Guadalajara
g_vars_5 = LpVariable('gf', cat='Binary' ) 

# The objective function is added to prob first
prob_5 += lpSum([route_vars[p][d]*distr_costs[p][d] for (p,d) in Routes]
              + [route_vars[p][d]*Prod_costs[p] for(p,d) in Routes]
              + [g_vars_5* 10000] ) # fixed cost per day if opening Guadalajara factory


# The capacity maximum constraints are added to prob for each supply node  
for p in Plants[:-1]:
   
    prob_5 += lpSum([route_vars[p][d] for d in DC]) <= Capacity[p]


prob_5 += lpSum([route_vars['5'][d] for d in DC]) <= Capacity['5']*g_vars_5

#The demand minimum constraints are added to prob for each demand node
for d in DC:
    prob_5 += lpSum([route_vars[p][d] for p in Plants]) == Demand_10[d]



# solve the model
prob_5.solve()
print ('Status:', LpStatus[prob_5.status])

# print objective function value
obj_5 = value(prob_5.objective)
print("The minimum total cost is: ${}".format(round(obj_5,2)))

# print binary value whether to open factory in Guadalajara (1 = Open; 0 = Not Open)
print('Whether to open Guadalajara factory:', round(g_vars_5.varValue))

# Print optimal QTY from each factory to distribution center
v_soln_5 = np.array([[round(route_vars[p][d].varValue) for d in DC] for p in Plants])
v_soln_df_5 = pd.DataFrame(v_soln_5)
v_soln_df_5.columns=['Toronto', 'Kansas City', 'Los Angeles', 'Seattle', 'Chicago', 'Atlanta', 'Guadalajara']
v_soln_df_5.index=['Toronto', 'Kansas City','Los Angeles', 'Seattle', 'Guadalajara']
v_soln_df_5



Status: Optimal
The minimum total cost is: $196500.0
Whether to open Guadalajara factory: 1


,Toronto,Kansas City,Los Angeles,Seattle,Chicago,Atlanta,Guadalajara
Toronto,1000,0,0,0,1500,0,0
Kansas City,0,500,0,0,0,0,0
Los Angeles,0,500,3000,0,0,0,0
Seattle,0,0,0,2000,500,0,0
Guadalajara,0,0,0,0,0,1000,3000


<span style='font-size:24px'><b>Scenario 6</b></span><br>
<b>*Run 90% Capacity in current facilities <mark>WITH</mark> Guadalajara Factory for the <mark>Tenth</mark> Year Demand*</b>

In [23]:
# Creates the prob variable to contain the problem data
prob_6 = LpProblem("CP_Problem_6", LpMinimize)
 
# Creates a list of tuples containing all the possible routes from factories to Distribution Centers
Routes = [(p,d) for p in Plants for d in DC]

# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route",(Plants,DC),0,None,LpInteger)


# Set up binary variable whether to open factory in Guadalajara
g_vars_6 = LpVariable('gf', cat='Binary' ) 

# The objective function is added to prob first
prob_6 += lpSum([route_vars[p][d]*distr_costs[p][d] for (p,d) in Routes]
              + [route_vars[p][d]*Prod_costs[p] for(p,d) in Routes]
              + [g_vars_6* 10000] ) # fixed cost per day if opening Guadalajara factory


# The capacity maximum constraints are added to prob for each supply node 
for p in Plants[:-1]:
   
    prob_6 += lpSum([route_vars[p][d] for d in DC]) <= Capacity[p]*0.9


prob_6 += lpSum([route_vars['5'][d] for d in DC]) <= Capacity['5']*g_vars_6

#The demand minimum constraints are added to prob for each demand node 
for d in DC:
    prob_6 += lpSum([route_vars[p][d] for p in Plants]) == Demand_10[d]



# solve the model
prob_6.solve()
print ('Status:', LpStatus[prob_6.status])

# print objective function value
obj_6 = value(prob_6.objective)
print("The minimum total cost is: ${}".format(round(obj_6,2)))

# print binary value whether to open factory in Guadalajara (1 - Open; 0 - Not Open)
print('Whether to open Guadalajara factory:', round(g_vars_6.varValue))

# Print optimal QTY from each factory to distribution center
v_soln_6 = np.array([[round(route_vars[p][d].varValue) for d in DC] for p in Plants])
v_soln_df_6 = pd.DataFrame(v_soln_6)
v_soln_df_6.columns=['Toronto', 'Kansas City', 'Los Angeles', 'Seattle', 'Chicago', 'Atlanta', 'Guadalajara']
v_soln_df_6.index=['Toronto', 'Kansas City','Los Angeles', 'Seattle', 'Guadalajara']
v_soln_df_6



Status: Optimal
The minimum total cost is: $199575.0
Whether to open Guadalajara factory: 1


,Toronto,Kansas City,Los Angeles,Seattle,Chicago,Atlanta,Guadalajara
Toronto,1000,0,0,0,1250,0,0
Kansas City,0,850,0,0,500,0,0
Los Angeles,0,150,3000,0,0,0,0
Seattle,0,0,0,2000,250,0,0
Guadalajara,0,0,0,0,0,1000,3000


<span style='font-size:24px'><b>Conclusion:</b></span>
<br>
<br>
Mapleleaf should take action to prepare new plant in Guadalajara next year, since total supply shortage would fall short of the growing demand in less than five year. The cost is substantial, about 6.2 million ((164562.5-143875)X 300days), for the new plant to be opened in Guadalajara. Meanwhile it could close factory in Kansas City to minimize cost as suggested under scenario 3 & scenario 4.